In [4]:
# Install Required Packages if you don't have them:
# !pip install python-dotenv joblib anthropic hume-api

# Import Statements
import os
from dotenv import load_dotenv
from joblib import load
from pprint import pprint
import time
import traceback

# Custom Modules (Assuming these are in your project structure)
from src.utils.anthropicwrapper import ClaudeChat, ClaudeChatAssess
from src.utils.humewrapper import HumeSentimentAnalyzer
from src.modules import ConversationVerifier

import json
import os

In [5]:

# --- Load Environment Variables ---
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))

# --- Timestamp Input ---
timestamp = 1725232852  # Example timestamp, replace with your input 

# --- Load Data ---
directory = f'data/interviews/{timestamp}/'
chatlog = load(os.path.join(directory, "joblib/conversation.joblib"))



FileNotFoundError: [Errno 2] No such file or directory: 'data/interviews/1725232852/joblib/conversation.joblib'

In [6]:
# --- Model Setup --- 
try:
    sentiment_analyser = HumeSentimentAnalyzer(api_key=os.getenv("HUME_API_KEY"))

    sentiment_system_prompt = """You are a skilled emotions analyst provided with a detailed breakdown of sentiment analysis scores from Hume.ai, for a single response in an interview to a question from the interviewer. The scores are split into 3 sections. All numbers are from 0 to 1, linearly scaling, with 1 being a very strong representation of the indicator in question.

    First, Emotions. This contains several human emotions with a numerical value indicating the strength of the corresponding emotion.
    Second, Sentiments. This contains a scale from 1 to 9, each containing a numerical value indicating the magnitude of the sentiment of the topic of the conversation. A negative topic such as murder will have a high value lower in the scale, such as 1 or 2, and a positive topic will have a high value from 0 to 1 higher in the scale such as 8 or 9.
    Third, Toxicity. This contains several toxic representations such as hate, insult, etc, with a value from 0 to 1 for each representation identified in the audio.

    Your job is to provide a concise detailed one sentence breakdown of how the individual was feeling for the particular scores provided. You must be highly objective as your job is to discern whether or not a candidate was exhibiting traits which would or would not be fitting for a successful interview. 
    Model your answer beginning with something along the lines of "For this particular response, the candidate..."""
    
    sentiment_summariser = ClaudeChat("claude-3-5-sonnet-20240620", sentiment_system_prompt)

    evaluation_system_prompt = """You are a highly skilled interviewer currently tasked with reviewing a phone screening interview candidate to decide whether they are to pass on to the next stage of the interview process. There is a high volume of candidates and as such only 47% of candidates should be allowed to pass.
    You will be provided with the following information to aid your decision:
    1. A copy of the job description.
    2. Transcript of the phone interview between the interviewer and candidate.
    3. Sentiment analysis summary to provide you with insight into the tone of the candidate in the call
    4. A summary of any factual inaccuracies the candidate may have made during discussions in the call
    5. A copy of the candidate's CV, which will be provided next.

    You are to evaluate the candidate, primarily on the transcript, and use the additional information provided to identify any potential red-flags. Your response should include a detailed breakdown of why the candidate is chosen to continue onwards to further interviewing. You must end the breakdown with a simple one word response on a new line, "pass" or "fail"."""

    candidate_evaluator = ClaudeChatAssess("claude-3-5-sonnet-20240620", evaluation_system_prompt, "data/cvs/cv-deb.pdf")

except Exception as e:
    print(f"An error occurred during model setup: {e}")
    raise



Error processing PDF: no such file: 'data/cvs/cv-deb.pdf'


In [7]:
# --- Reformat Chatlog ---
def reformat_chatlog(chatlog):
    dropped_context = chatlog[3:]
    outputchatlog = []

    for i in range(0, len(dropped_context), 2):
        if i + 1 < len(dropped_context):
            tempdict = {
                'interviewer': dropped_context[i]['content'],
                'candidate': dropped_context[i+1]['content']
            }
            outputchatlog.append(tempdict)
        else:
            break 
            
    return outputchatlog

# --- Process Sentiments ---


In [8]:
def process_sentiments(chatlog_chat, timestamp):
    filepath = os.path.join('data', 'interviews', str(timestamp), 'audio')
    print(f"Current working directory: {os.getcwd()}")
    print(f"Full audio directory path: {os.path.abspath(filepath)}")

    files = [f for f in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, f))]
    
    if len(chatlog_chat) < len(files):
        files = files[:len(chatlog_chat)]

    for f in files:
        print(f"File found: {f}")

    sentiments = []
    
    for count, file in enumerate(files, 1):
        full_file_path = os.path.join(filepath, file)
        print(f"Processing file: {full_file_path}")
        
        # Add a small delay and re-check file existence
        time.sleep(0.1)
        if not os.path.exists(full_file_path):
            print(f"File not found (after delay): {full_file_path}")
            continue

        try:
            result = sentiment_analyser.analyze_audio(full_file_path)
            sentiment_summary = sentiment_summariser.chat(str(result))
            sentiments.append((result, sentiment_summary))
            chatlog_chat[count-1]['sentiment'] = sentiment_summary
        except Exception as e:
            print(f"Error processing file {full_file_path}: {str(e)}")
            traceback.print_exc()

    return chatlog_chat



In [9]:
# --- Evaluate Candidate --- 
def evaluate_candidate(chatlog_chat):
    ConversationVerifier.process_qa_pair(chatlog_chat)
    print("The Feedback JSON from the sentiment analyser and accuracy verifier: \n")
    pprint(chatlog_chat)
    evaluation = candidate_evaluator.chat(str(chatlog_chat))
    return evaluation



In [19]:
    # --- Main Execution Flow ---
for i in range(1):
        print(f'---------------------------{timestamp}-{i}------------------------')
        chatlog_chat = reformat_chatlog(chatlog)  # Process the loaded chatlog
        chatlog_chat = process_sentiments(chatlog_chat, timestamp) # Analyze sentiments

        #chatlog_chat_copy = chatlog_chat[1]
        evaluation = evaluate_candidate(chatlog_chat) # Get the final evaluation


        chatlog_file_path = f"data/interviews/{timestamp}-{i}/outcome/"

        if not os.path.exists(chatlog_file_path):
            os.makedirs(chatlog_file_path)

        with open(chatlog_file_path+"chatlog.json", "w") as file:
            json.dump(chatlog_chat, file, indent=4)
        evaluation_file_path = f"data/interviews/{timestamp}-{i}/outcome/"
        if not os.path.exists(chatlog_file_path):
            os.makedirs(chatlog_file_path)
        with open(evaluation_file_path+"evaluation.txt", "w") as file:
            file.write(str(evaluation).replace("\\n", "\n"))
        print(f'---------------------------{timestamp}-{i}------------------------')


---------------------------1725380720-0------------------------
Current working directory: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM
Full audio directory path: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\data\interviews\1725380720\audio
attempting to generate searc queries from answers
No documents retrieved from webscrapping 

Feedback is being returned from accuracy verifier
The Feedback JSON from the sentiment analyser and accuracy verifier: 

[]
---------------------------1725380720-0------------------------


In [9]:

print(f'---------------------------{timestamp}------------------------')
chatlog_chat = reformat_chatlog(chatlog)  # Process the loaded chatlog
chatlog_chat = process_sentiments(chatlog_chat, timestamp) # Analyze sentiments

#chatlog_chat_copy = chatlog_chat[1]
evaluation = evaluate_candidate(chatlog_chat) # Get the final evaluation


chatlog_file_path = f"data/interviews/{timestamp}/outcome/"

if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)

with open(chatlog_file_path+"chatlog.json", "w") as file:
    json.dump(chatlog_chat, file, indent=4)
evaluation_file_path = f"data/interviews/{timestamp}/outcome/"
if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)
with open(evaluation_file_path+"evaluation.txt", "w") as file:
    file.write(str(evaluation).replace("\\n", "\n"))
print(f'---------------------------{timestamp}------------------------')

---------------------------1725236280------------------------
Current working directory: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM
Full audio directory path: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\data\interviews\1725236280\audio
File found: audio_1_1725236280.wav
Processing file: data\interviews\1725236280\audio\audio_1_1725236280.wav


Analyzing audio file: data\interviews\1725236280\audio\audio_1_1725236280.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_1_1725236280.wav', 'content_type': 'audio/wav', 'md5sum': '64933daa432855048bca1d9b44268a8b'}, 'results': {'predictions': [{'file': 'audio_1_1725236280.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.79833984, 'detected_language': 'hi'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': 'Hello.', 'position': {'begin': 0, 'end': 6}, 'time': {'begin': 0.62, 'end': 0.74}, 'confidence': 0.99072266, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.01251170877367258}, {'name': 'Adoration', 'score': 0.008131410926580429}, {'name': 'Aesthetic Appreciation', 'score': 0.016512403264641762}, {'name': 'Amusement', 'score': 0.0907270684838295}, {'name': 'Anger', 'score': 0.0005581923178397119}, {'name': 'Annoyance', 'score': 0.014846810139715

KeyboardInterrupt: 

In [12]:
chatlog_chat = [
    {
        "interviewer": "Hello! Thank you for taking the time to speak with me today about the Entry Level RAG Engineer position. I'd like to ask you a few questions to learn more about your experience and skills. Could you tell me about your experience with retrieval-augmented generation (RAG) pipelines?",
        "candidate": " Yes, sure. I have some experience in retrieval of augmented generation pipelines. Mainly I used RAG to do my final thesis project, which is an automated screening system. I used RAG for the verification process to put up-to-date information from the internet in the context of an LLM so that it can verify all the information that is available currently.",
        "sentiment": "For this particular response, the candidate exhibited a balanced mix of emotions, with notable levels of calmness (0.32), interest (0.22), and positive surprise (0.16), alongside moderate levels of boredom (0.11) and enthusiasm (0.09), suggesting an engaged yet somewhat reserved demeanor during the interview.",
        "feedback": "The candidate is well versed in all aspects of the question asked"
    },
    {
        "interviewer": "That's interesting. Can you elaborate on the specific challenges you faced while implementing RAG in your thesis project, and how you overcame them?",
        "candidate": " Sure, so breaking down a candidate's answer into queries wasn't my first approach. I couldn't find a way to break down the answer or like provide a search query for getting information on the relevant topics and then through query decomposition I got relevant answers like by breaking down the string into sub queries and then searching for it. That expanded the search space so that is how I solved it.",
        "sentiment": "For this particular response, the candidate exhibited a high level of concentration (0.70) and determination (0.39), coupled with significant feelings of triumph (0.47) and satisfaction (0.35), suggesting a focused and confident demeanor during the interview, while maintaining a generally neutral sentiment (0.89 in sentiment level 5) and showing no signs of toxicity, which are all positive indicators for a successful interview performance.",
        "feedback": "The candidate is well versed in all aspects of the question asked"
    },
    {
        "interviewer": "Thank you for sharing that. Moving on, can you describe your experience with different large language models (LLMs) and how you've compared their performance for specific tasks?",
        "candidate": " I'm sure I've worked with JadGPT, Claude, Sonnet and Lama. I would rank them as Claude Sonnet is the highest for reasoning, JadGPT comes in second and Lama comes in third.",
        "sentiment": "For this particular response, the candidate exhibited a high level of concentration (0.39) and satisfaction (0.40), coupled with moderate levels of calmness (0.20) and contentment (0.19), suggesting a focused and positive demeanor during the interview, while also displaying some interest (0.15) and determination (0.15) which are favorable traits in an interview setting.",
        "feedback": "The candidate is well versed in all aspects of the question asked"
    }
]

In [13]:
timestamp = "1725380262-rec-man-pass"
#evaluation = evaluate_candidate(chatlog_chat) # Get the final evaluation
evaluation = candidate_evaluator.chat(str(chatlog_chat))
    
chatlog_file_path = f"data/interviews/{timestamp}/outcome/"

if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)

with open(chatlog_file_path+"chatlog.json", "w") as file:
    json.dump(chatlog_chat, file, indent=4)
evaluation_file_path = f"data/interviews/{timestamp}/outcome/"
if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)
with open(evaluation_file_path+"evaluation.txt", "w") as file:
    file.write(str(evaluation).replace("\\n", "\n"))
print(f'---------------------------{timestamp}------------------------')


---------------------------1725380262-rec-man-pass------------------------
